**You may need to install [OpenCV](https://pypi.python.org/pypi/opencv-python) and [scikit-video](http://www.scikit-video.org/stable/).**

In [1]:
import keras
import numpy as np
import io
import base64
from IPython.display import HTML
import skvideo.io
import cv2
import json

from keras.models import Sequential,model_from_json
from keras.layers.core import Dense
from keras.optimizers import sgd
from keras.layers import Conv2D, MaxPooling2D, Activation, AveragePooling2D,Reshape,BatchNormalization

Using TensorFlow backend.


# MiniProject #3: Deep Reinforcement Learning

__Notations__: $E_p$ is the expectation under probability $p$. Please justify each of your answer and widely comment your code.

# Context

In a reinforcement learning algorithm, we modelize each step $t$ as an action $a_t$ obtained from a state $s_t$, i.e. $\{(a_{t},s_{t})_{t\leq T}\}$ having the Markov property. We consider a discount factor $\gamma \in [0,1]$ that ensures convergence. The goal is to find among all the policies $\pi$, one that maximizes the expected reward:

\begin{equation*}
R(\pi)=\sum_{t\leq T}E_{p^{\pi}}[\gamma^t r(s_{t},a_{t})] \> ,
\end{equation*}

where: 
\begin{equation*}p^{\pi}(a_{0},a_{1},s_{1},...,a_{T},s_{T})=p(a_{0})\prod_{t=1}^{T}\pi(a_{t}|s_{t})p(s_{t+1}|s_{t},a_{t}) \> .
\end{equation*}

We note the $Q$-function:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq T}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

Thus, the optimal Q function is:
\begin{equation*}
Q^*(s,a)=\max_{\pi}Q^\pi(s,a) \> .
\end{equation*}

In this project, we will apply the deep reinforcement learning techniques to a simple game: an agent will have to learn from scratch a policy that will permit it maximizing a reward.

## The environment, the agent and the game

### The environment

```Environment``` is an abstract class that represents the states, rewards, and actions to obtain the new state.

In [2]:
class Environment(object):
    def __init__(self):
        pass

    def act(self, act):
        """
        One can act on the environment and obtain its reaction:
        - the new state
        - the reward of the new state
        - should we continue the game?

        :return: state, reward, game_over
        """
        pass


    def reset(self):
        """
        Reinitialize the environment to a random state and returns
        the original state

        :return: state
        """
        pass
    
    def draw(self):
        """
        Visualize in the console or graphically the current state
        """
        pass

The method ```act``` allows to act on the environment at a given state $s_t$ (stored internally), via action $a_t$. The method will return the new state $s_{t+1}$, the reward $r(s_{t},a_{t})$ and determines if $t\leq T$ (*game_over*).

The method ```reset``` simply reinitializes the environment to a random state $s_0$.

The method ```draw``` displays the current state $s_t$ (this is useful to check the behavior of the Agent).

We modelize $s_t$ as a tensor, while $a_t$ is an integer.

### The Agent

The goal of the ```Agent``` is to interact with the ```Environment``` by proposing actions $a_t$ obtained from a given state $s_t$ to attempt to maximize its __reward__ $r(s_t,a_t)$. We propose the following abstract class:

In [3]:
class Agent(object):
    def __init__(self, epsilon=0.1, n_action=4):
        self.epsilon = epsilon
        self.n_action = n_action
    
    def set_epsilon(self, e):
        self.epsilon = e

    def act(self, s, train=True):
        """ This function should return the next action to do:
        an integer between 0 and 4 (not included) with a random exploration of epsilon"""
        if train:
            if np.random.rand() <= self.epsilon:
                a = np.random.randint(0, self.n_action, size=1)[0]
            else:
                a = self.learned_act(s)
        else: # in some cases, this can improve the performance.. remove it if poor performances
            a = self.learned_act(s)

        return a

    def learned_act(self, s):
        """ Act via the policy of the agent, from a given state s
        it proposes an action a"""
        pass

    def reinforce(self, s, n_s, a, r, game_over_):
        """ This function is the core of the learning algorithm. 
        It takes as an input the current state s_, the next state n_s_
        the action a_ used to move from s_ to n_s_ and the reward r_.
        
        Its goal is to learn a policy.
        """
        pass

    def save(self):
        """ This function returns basic stats if applicable: the
        loss and/or the model"""
        pass

    def load(self):
        """ This function allows to restore a model"""
        pass

***
__Question 1__:
Explain the function act. Why is ```epsilon``` essential?

The goal is to choose and action according to what the agent has learned until now. This is why we are returning `self.learned_act(s)`. But there is an `epsilon` chance that we return a random action. Indeed, as the saying goes: "only fools never change their minds", so the agent should try something new in order to explore the state-action space and improve it's policy.

***
### The Game

The ```Agent``` and the ```Environment``` work in an interlaced way as in the following (take some time to understand this code as it is the core of the project)

```python

epoch = 300
env = Environment()
agent = Agent()


# Number of won games
score = 0
loss = 0


for e in range(epoch):
    # At each epoch, we restart to a fresh game and get the initial state
    state = env.reset()
    # This assumes that the games will end
    game_over = False

    win = 0
    lose = 0
    
    while not game_over:
        # The agent performs an action
        action = agent.act(state)

        # Apply an action to the environment, get the next state, the reward
        # and if the games end
        prev_state = state
        state, reward, game_over = env.act(action)

        # Update the counters
        if reward > 0:
            win = win + reward
        if reward < 0:
            lose = lose -reward

        # Apply the reinforcement strategy
        loss = agent.reinforce(prev_state, state,  action, reward, game_over)

    # Save as a mp4
    if e % 10 == 0:
        env.draw(e)

    # Update stats
    score += win-lose

    print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
          .format(e, epoch, loss, win, lose, win-lose))
    agent.save()
```

# The game, *eat cheese*

A rat runs on an island and tries to eat as much as possible. The island is subdivided into $N\times N$ cells, in which there are cheese (+0.5) and poisonous cells (-1). The rat has a visibility of 2 cells (thus it can see $5^2$ cells). The rat is given a time $T$ to accumulate as much food as possible. It can perform 4 actions: going up, down, left, right. 

The goal is to code an agent to solve this task that will learn by trial and error. We propose the following environment:

In [4]:
class Environment(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size + 4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size, grid_size))
        self.position = np.zeros((grid_size, grid_size))

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time + 2, grid_size * self.scale, grid_size * self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size, self.grid_size,3)) + 128
        b[self.board > 0, 0] = 256
        b[self.board < 0, 2] = 256
        b[self.x, self.y, :] = 256
        b[-2:, :, :] = 0
        b[:, -2:, :] = 0
        b[:2, :, :] = 0
        b[:, :2, :] = 0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t, :, :, :] = b


    def act(self, action):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')

        self.t = self.t + 1
        reward = self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size,1),
                self.position.reshape(self.grid_size, self.grid_size,1)
            ),
            axis=2
        )
        state = state[self.x - 2: self.x + 3, self.y - 2: self.y + 3, :]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]


        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size ** 2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time + 2, self.grid_size * self.scale, self.grid_size * self.scale, 3))


        malus[bonus > 0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate(
            (
                self.board.reshape(self.grid_size, self.grid_size,1),
                self.position.reshape(self.grid_size, self.grid_size,1)
            ),
            axis=2
        )

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

The following elements are important because they correspond to the hyper parameters for this project:

In [5]:
# parameters
size = 13
T = 200
temperature = 0.3
epochs_train = 30 # set small when debugging
epochs_test = 30 # set small when debugging

# display videos
def display_videos(name):
    video = io.open(name, 'r+b').read()
    encoded = base64.b64encode(video)
    return '''<video alt="test" controls>
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))

__Question 2__ Explain the use of the arrays ```position``` and ```board```.

The board attribute reprensents the rewards displayed on the grid (cheese and poison), as we can see in the `reset` function. We can see that the rewards are stored in the board attribute in the `act` method: `reward = self.board[self.x, self.y]` and this line `self.board[self.x, self.y] = 0` shows that once an object has been consumed, it disappears.

It looks like the agent is getting two different information about the board (he is returned two $5*5$ arrays). First, he is getting information about his position, he is at the center of the array, his previous position is marked by a 1. All other positions are marked as 0 except if they are on the "border", then they are marked as -1. This is important for the agent, since he cannot cross the border, if he attempts to do it, his position will be moved in the opposed direction. This way he can understand when he is heading towards a border and if he should avoid it.

## Random Agent

***
__Question 3__ Implement a random Agent (only ```learned_act``` needs to be implemented):

In [6]:
class RandomAgent(Agent):
    def __init__(self):
        super(RandomAgent, self).__init__()
        pass

    def learned_act(self, s):
        a = np.random.randint(0, self.n_action, size=1)[0]
        return a

***
***
__Question 4__ Visualize the game moves. You need to fill in the following function for the evaluation:

In [7]:
def test(agent, env, epochs, prefix=''):
    # Number of won games
    score = 0
        
    for e in range(epochs):
        state = env.reset()
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action)

            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose - reward

        env.draw(prefix + str(e))

        score += win-lose
        
        print("Win/lose count {}/{}. Average score ({})"
              .format(win, lose, score / (1 + e)))
    print('Final score: ' + str(score / epochs))

In [14]:
# Initialize the game
env = Environment(grid_size=size, max_time=T, temperature=temperature)

# Initialize the agent!
agent = RandomAgent()

test(agent,env ,epochs_test, prefix='random')
HTML(display_videos('random0.mp4'))

Win/lose count 9.0/12.0. Average score (-3.0)
Win/lose count 11.0/16.0. Average score (-4.0)
Win/lose count 10.5/11.0. Average score (-2.8333333333333335)
Win/lose count 8.5/10.0. Average score (-2.5)
Win/lose count 10.0/15.0. Average score (-3.0)
Win/lose count 9.5/14.0. Average score (-3.25)
Win/lose count 10.0/9.0. Average score (-2.642857142857143)
Win/lose count 9.0/10.0. Average score (-2.4375)
Win/lose count 13.0/20.0. Average score (-2.9444444444444446)
Win/lose count 6.0/9.0. Average score (-2.95)
Win/lose count 9.0/11.0. Average score (-2.8636363636363638)
Win/lose count 7.0/18.0. Average score (-3.5416666666666665)
Win/lose count 9.5/11.0. Average score (-3.3846153846153846)
Win/lose count 11.0/15.0. Average score (-3.4285714285714284)
Win/lose count 9.5/12.0. Average score (-3.3666666666666667)
Win/lose count 14.5/13.0. Average score (-3.0625)
Win/lose count 12.0/19.0. Average score (-3.2941176470588234)
Win/lose count 10.5/15.0. Average score (-3.361111111111111)
Win/lose 

***
## DQN

Let us assume here that $T=\infty$.

***
__Question 5__ Let $\pi$ be a policy, show that:

\begin{equation*}
Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]
\end{equation*}

Then, show that for the optimal policy $\pi^*$ (we assume its existence), the following holds: 

\begin{equation*}
Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')].
\end{equation*}
Finally, deduce that a plausible objective is:

\begin{equation*}
\mathcal{L}(\theta)=E_{s' \sim \pi^*(.|s,a)}\Vert r+\gamma\max\max_{a'}Q(s',a',\theta)-Q(s,a,\theta)\Vert^{2}.
\end{equation*}




## my answer:

\begin{equation*}Q^\pi(s,a)=E_{p^{\pi}}[\sum_{t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a] \> .
\end{equation*}

$$ Q^\pi(s,a)=r(s, a) + E_{p^{\pi}}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{0}=s,a_{0}=a]$$

$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']|s_0=s, a_0=a]$$

$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{1\leq t\leq \infty}\gamma^{t}r(s_{t},a_{t})|s_{1}=s',a_{1}=a']]$$

$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[E_{p^\pi}[\sum_{0\leq t\leq \infty}\gamma^{t+1}r(s_{t},a_{t})|s_{0}=s',a_{0}=a']]$$

$$ Q^\pi(s,a)=r(s, a) + E_{(s',a')\sim p(.|s,a)}[\gamma * Q^{\pi}(s', a')]$$

$$Q^{\pi}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{\pi}(s',a')]$$

From the first question, we have:
$$Q^{*}(s,a)=E_{(s',a')\sim p(.|s,a)}[r(s,a)+\gamma Q^{*}(s',a')]$$

Since Q* is optimal, it must be that $\pi^{*}(s') = argmax(Q^{*}(s', .))$ with probability $1$. Or else the agent having the exact same policy as $\pi^{*}$ but with $\pi^{*}(s') = argmax(Q^{*}(s', .))$ with probability $1$ would be better than the optimal agent. Therefore:
$$Q^{*}(s,a)=E_{s'\sim \pi^*(.|s,a)}[r(s,a)+\gamma\max_{a'}Q^{*}(s',a')]$$

Since the above equation is respected by the optimal agent and we want to approach this agent, our agent should respect the equation too. Fixing this objective $\mathcal{L}(\theta)$ seems like a reasonnable approach.

***
The DQN-learning algorithm relies on these derivations to train the parameters $\theta$ of a Deep Neural Network:

1. At the state $s_t$, select the action $a_t$ with best reward using $Q_t$ and store the results;

2. Obtain the new state $s_{t+1}$ from the environment $p$;

3. Store $(s_t,a_t,s_{t+1})$;

4. Obtain $Q_{t+1}$ by minimizing  $\mathcal{L}$ from a recovered batch from the previously stored results.

***
__Question 6__ Implement the class ```Memory``` that stores moves (in a replay buffer) via ```remember``` and provides a ```random_access``` to these. Specify a maximum memory size to avoid side effects. You can for example use a ```list()``` and set by default ```max_memory=100```.

In [8]:
class Memory(object):
    def __init__(self, max_memory=100, batch_size=64):
        self.max_memory = max_memory
        self.memory = list()
        self.batch_size = batch_size

    def remember(self, m):
        self.memory.append(m)
        if len(self.memory) > self.max_memory:
            self.memory.pop(0)

    def random_access(self):
        choice = np.random.choice(range(len(self.memory)))
        chosen = self.memory[choice]
        return chosen

***
The pipeline we will use for training is given below:

In [9]:
def train(agent,env,epoch,prefix=''):
    # Number of won games
    score = 0
    loss = 0

    for e in range(epoch):
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            # The agent performs an action
            action = agent.act(state)
            
            # Apply an action to the environment, get the next state, the reward
            # and if the games end
            prev_state = state
            state, reward, game_over = env.act(action)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')

***
__Question 7__ Implement the DQN training algorithm using a cascade of fully connected layers. You can use different learning rate, batch size or memory size parameters. In particular, the loss might oscillate while the player will start to win the games. You have to find a good criterium.

In [16]:
class DQN(Agent):
    def __init__(self, grid_size, epsilon = 0.1, memory_size=100, batch_size = 16,n_state=2):
        super(DQN, self).__init__(epsilon=epsilon)

        # Discount for Q learning
        self.discount = 0.9
        
        self.grid_size = grid_size
        
        # number of state
        self.n_state = n_state

        # Memory
        self.memory = Memory(memory_size)
        
        # Batch size when learning
        self.batch_size = batch_size

    def learned_act(self, s):
        return np.argmax(self.model.predict(np.expand_dims(s, axis=0))[0])

    def reinforce(self, s_, n_s_, a_, r_, game_over_):
        # Two steps: first memorize the states, second learn from the pool

        self.memory.remember([s_, n_s_, a_, r_, game_over_])
        
        input_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        new_states = np.zeros((self.batch_size, 5, 5, self.n_state))
        target_q = np.zeros((self.batch_size, 4))
        actions = np.zeros(self.batch_size).astype(int)
        rewards = np.zeros(self.batch_size)

        for i in range(self.batch_size):
            s, n_s, a, r, game_over = self.memory.random_access()
            input_states[i, :, :, :] = s
            new_states[i, :, :, :] = n_s
            actions[i] = a
            rewards[i] = r

        target_q = self.model.predict(input_states)
        next_q = self.model.predict(new_states)
        best_q = np.max(next_q, axis=1)
        target_q[np.arange(self.batch_size), actions] = rewards + self.discount * best_q

        ######## FILL IN
        # HINT: Clip the target to avoid exploiding gradients.. -- clipping is a bit tighter
        target_q = np.clip(target_q, -3, 3)

        start = time.time()
        l = self.model.train_on_batch(input_states, target_q)
        end = time.time()
        return l

    def save(self,name_weights='model.h5',name_model='model.json'):
        self.model.save_weights(name_weights, overwrite=True)
        with open(name_model, "w") as outfile:
            json.dump(self.model.to_json(), outfile)
            
    def load(self,name_weights='model.h5',name_model='model.json'):
        with open(name_model, "r") as jfile:
            model = model_from_json(json.load(jfile))
        model.load_weights(name_weights)
        model.compile("sgd", "mse")
        self.model = model

from keras.models import Model
from keras.layers import Input, Dense, Flatten

class DQN_FC(DQN):
    def __init__(self, *args, lr=0.1,**kwargs):
        super(DQN_FC, self).__init__( *args,**kwargs)
        
        # NN Model
        model = Sequential()
                
        a = Input(shape=(5, 5, self.n_state, ))
        b = Flatten()(a)
        c = Dense(4)(b)
        model = Model(inputs=a, outputs=c)
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

In [17]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_FC(size, lr=.1, epsilon=0.1, memory_size=2000, batch_size=32)
epochs_train = 50
train(agent, env, epochs_train, prefix='fc_train')
HTML(display_videos('fc_train10.mp4'))

Epoch 000/050 | Loss 0.0096 | Win/lose count 3.5/5.0 (-1.5)
Epoch 001/050 | Loss 0.0162 | Win/lose count 5.0/6.0 (-1.0)
Epoch 002/050 | Loss 0.0171 | Win/lose count 4.0/6.0 (-2.0)
Epoch 003/050 | Loss 0.0160 | Win/lose count 8.0/5.0 (3.0)
Epoch 004/050 | Loss 0.0175 | Win/lose count 5.0/9.0 (-4.0)
Epoch 005/050 | Loss 0.0223 | Win/lose count 10.0/2.0 (8.0)
Epoch 006/050 | Loss 0.0087 | Win/lose count 5.5/4.0 (1.5)
Epoch 007/050 | Loss 0.0062 | Win/lose count 6.0/3.0 (3.0)
Epoch 008/050 | Loss 0.0095 | Win/lose count 9.0/4.0 (5.0)
Epoch 009/050 | Loss 0.0152 | Win/lose count 5.5/1.0 (4.5)
Epoch 010/050 | Loss 0.0045 | Win/lose count 3.5/2.0 (1.5)
Epoch 011/050 | Loss 0.0025 | Win/lose count 8.0/4.0 (4.0)
Epoch 012/050 | Loss 0.0075 | Win/lose count 9.0/7.0 (2.0)
Epoch 013/050 | Loss 0.0076 | Win/lose count 2.0/3.0 (-1.0)
Epoch 014/050 | Loss 0.0053 | Win/lose count 4.5/5.0 (-0.5)
Epoch 015/050 | Loss 0.0094 | Win/lose count 9.0/1.0 (8.0)
Epoch 016/050 | Loss 0.0022 | Win/lose count 4.5/

As we can see in this video, since he is shortsighted at some point he doesn't see any cheese and he gets blocked in a left->right->left cycle. We will see in the next part how we can change this by adding a reward for exploration of new cells.

***
***
__Question 8__ Implement the DQN training algorithm using a CNN (for example, 2 convolutional layers and one final fully connected layer).

In [19]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense


class DQN_CNN(DQN):
    def __init__(self, *args,lr=0.1,**kwargs):
        super(DQN_CNN, self).__init__(*args,**kwargs)
        
        model = Sequential()
        
        model.add(Conv2D(8, (2, 2), strides=1, input_shape=(5, 5, self.n_state),  padding="valid"))
        model.add(Activation('relu'))
        
        model.add(Conv2D(8, (2, 2), strides=1, input_shape=(5, 5, self.n_state),  padding="valid"))
        model.add(Activation('relu'))
        
        model.add(Flatten())
        model.add(Dense(4))
        
        model.compile(sgd(lr=lr, decay=1e-4, momentum=0.0), "mse")
        self.model = model

We propose a 2 layers convolutionnal network with (2*2) filters, and a last fully connected layer. We could exploit the symmetries if we made a model predicting the Q-value of going up, then to predict the Q-value of going left, we can just rotate the grid, this way we have less parameters to learn and it garanties that the model is symmetric is every direction.

In [20]:
env = Environment(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=100000, batch_size = 32)
print(agent.model.summary())
train(agent,env,epochs_train,prefix='cnn_train')
HTML(display_videos('cnn_train10.mp4'))

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_3 (Conv2D)            (None, 4, 4, 8)           72        
_________________________________________________________________
activation_3 (Activation)    (None, 4, 4, 8)           0         
_________________________________________________________________
conv2d_4 (Conv2D)            (None, 3, 3, 8)           264       
_________________________________________________________________
activation_4 (Activation)    (None, 3, 3, 8)           0         
_________________________________________________________________
flatten_5 (Flatten)          (None, 72)                0         
_________________________________________________________________
dense_5 (Dense)              (None, 4)                 292       
Total params: 628
Trainable params: 628
Non-trainable params: 0
_________________________________________________________________
None
Epoch 0

***
***
__Question 9__ Test both algorithms and compare their performances. Which issue(s) do you observe? Observe also different behaviors by changing the temperature.

In [21]:
agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_trainmodel.h5',name_model='cnn_trainmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_trainmodel.h5',name_model='fc_trainmodel.json')
print('Test of the CNN')
np.random.seed(0)
env = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_cnn,env,epochs_test,prefix='cnn_test')
print('Test of the FC')
np.random.seed(0)
env = Environment(grid_size=size, max_time=T,temperature=0.3)
test(agent_fc,env,epochs_test,prefix='fc_test')

Test of the CNN
Win/lose count 11.5/1.0. Average score (10.5)
Win/lose count 19.0/2.0. Average score (13.75)
Win/lose count 5.0/0. Average score (10.833333333333334)
Win/lose count 11.5/2.0. Average score (10.5)
Win/lose count 15.5/2.0. Average score (11.1)
Win/lose count 5.5/1.0. Average score (10.0)
Win/lose count 10.0/2.0. Average score (9.714285714285714)
Win/lose count 13.5/1.0. Average score (10.0625)
Win/lose count 6.5/1.0. Average score (9.555555555555555)
Win/lose count 2.0/4.0. Average score (8.4)
Win/lose count 3.5/2.0. Average score (7.7727272727272725)
Win/lose count 10.5/2.0. Average score (7.833333333333333)
Win/lose count 8.5/4.0. Average score (7.576923076923077)
Win/lose count 14.5/3.0. Average score (7.857142857142857)
Win/lose count 17.0/3.0. Average score (8.266666666666667)
Win/lose count 11.5/3.0. Average score (8.28125)
Win/lose count 15.0/0. Average score (8.676470588235293)
Win/lose count 18.0/5.0. Average score (8.916666666666666)
Win/lose count 5.5/0. Averag

In [22]:
HTML(display_videos('cnn_test10.mp4'))

In [23]:
HTML(display_videos('fc_test10.mp4'))

We observe that the CNN model gets better results than the fully connected model (9.2 vs 8.2). We also observe that both model tend to get stuck in infinite loops.

Below we train and test both models with a higher temperature (more elements on the map).

In [24]:
temperature = 0.8

print("training hot FC:")
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='fc_train_hot')
HTML(display_videos('fc_train_hot10.mp4'))


print("training hot CNN:")
env = Environment(grid_size=size, max_time=T, temperature=temperature)
agent = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
train(agent, env, epochs_train, prefix='cnn_train_hot')
HTML(display_videos('cnn_train_hot10.mp4'))


agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_hotmodel.h5',name_model='cnn_train_hotmodel.json')

agent_fc = DQN_FC(size, lr=.1, epsilon = 0.1, memory_size=2000, batch_size = 32)
agent_fc.load(name_weights='fc_train_hotmodel.h5',name_model='fc_train_hotmodel.json')
print('Test of the HOT CNN')
np.random.seed(0)
env = Environment(grid_size=size, max_time=T,temperature=temperature)
test(agent_cnn, env,epochs_test, prefix='cnn_test_hot')
print('Test of the HOT FC')
np.random.seed(0)
env = Environment(grid_size=size, max_time=T,temperature=temperature)
test(agent_fc, env,epochs_test, prefix='fc_test_hot')

Epoch 000/050 | Loss 0.0195 | Win/lose count 17.5/7.0 (10.5)
Epoch 001/050 | Loss 0.0065 | Win/lose count 22.5/4.0 (18.5)
Epoch 002/050 | Loss 0.0207 | Win/lose count 13.0/4.0 (9.0)
Epoch 003/050 | Loss 0.0194 | Win/lose count 10.0/8.0 (2.0)
Epoch 004/050 | Loss 0.0154 | Win/lose count 13.0/0 (13.0)
Epoch 005/050 | Loss 0.0207 | Win/lose count 19.0/4.0 (15.0)
Epoch 006/050 | Loss 0.0186 | Win/lose count 16.5/5.0 (11.5)
Epoch 007/050 | Loss 0.0141 | Win/lose count 16.0/1.0 (15.0)
Epoch 008/050 | Loss 0.0096 | Win/lose count 27.5/3.0 (24.5)
Epoch 009/050 | Loss 0.0168 | Win/lose count 22.5/1.0 (21.5)
Epoch 010/050 | Loss 0.0177 | Win/lose count 27.5/2.0 (25.5)
Epoch 011/050 | Loss 0.0173 | Win/lose count 33.5/3.0 (30.5)
Epoch 012/050 | Loss 0.0179 | Win/lose count 34.5/3.0 (31.5)
Epoch 013/050 | Loss 0.0241 | Win/lose count 23.5/6.0 (17.5)
Epoch 014/050 | Loss 0.0207 | Win/lose count 30.5/4.0 (26.5)
Epoch 015/050 | Loss 0.0154 | Win/lose count 30.0/4.0 (26.0)
Epoch 016/050 | Loss 0.0297 

Win/lose count 59.0/1.0. Average score (48.7)
Win/lose count 56.5/0. Average score (49.40909090909091)
Win/lose count 61.5/2.0. Average score (50.25)
Win/lose count 23.5/1.0. Average score (48.11538461538461)
Win/lose count 59.0/2.0. Average score (48.75)
Win/lose count 54.5/2.0. Average score (49.0)
Win/lose count 55.5/1.0. Average score (49.34375)
Win/lose count 34.5/2.0. Average score (48.35294117647059)
Win/lose count 42.5/2.0. Average score (47.916666666666664)
Win/lose count 62.5/2.0. Average score (48.578947368421055)
Win/lose count 29.0/3.0. Average score (47.45)
Win/lose count 39.0/4.0. Average score (46.857142857142854)
Win/lose count 59.5/2.0. Average score (47.34090909090909)
Win/lose count 47.0/3.0. Average score (47.19565217391305)
Win/lose count 39.0/2.0. Average score (46.770833333333336)
Win/lose count 57.0/1.0. Average score (47.14)
Win/lose count 20.5/0. Average score (46.11538461538461)
Win/lose count 44.0/6.0. Average score (45.81481481481482)
Win/lose count 27.0/2

In [27]:
HTML(display_videos('cnn_test_hot10.mp4'))

In [28]:
HTML(display_videos('fc_test_hot10.mp4'))

Once again the CNN is better than the FC model (49.1 vs 45.1). From what we observe in the videos, the agent tends to explore more the map. It can be explained because temperature not only increases the number of elements on the map but it also increases the ratio (number of cheese / number of poison), so exploration of the map is very well rewarded. On the other hand, with a lower temperature the agent tends to focus on avoiding poisonous cells.

We can still observe that the agent is getting stuck in infinite loops.

What if we now test our agent but this time we disable the exploration ? In theory the agent should have better results:

In [25]:
# now we test again the CNN but with no exploration during the test phase
temperature = 0.8

agent_cnn = DQN_CNN(size, lr=.1, epsilon = 0., memory_size=2000, batch_size = 32)
agent_cnn.load(name_weights='cnn_train_hotmodel.h5',name_model='cnn_train_hotmodel.json')

print('Test of the HOT CNN')
np.random.seed(0)
env = Environment(grid_size=size, max_time=T,temperature=temperature)
test(agent_cnn, env,epochs_test, prefix='cnn_test_hot')

Test of the HOT CNN
Win/lose count 52.5/0. Average score (52.5)
Win/lose count 54.5/2.0. Average score (52.5)
Win/lose count 43.5/0. Average score (49.5)
Win/lose count 52.5/5.0. Average score (49.0)
Win/lose count 59.0/2.0. Average score (50.6)
Win/lose count 48.0/1.0. Average score (50.0)
Win/lose count 32.0/0. Average score (47.42857142857143)
Win/lose count 47.5/5.0. Average score (46.8125)
Win/lose count 53.0/0. Average score (47.5)
Win/lose count 44.5/0. Average score (47.2)
Win/lose count 57.0/2.0. Average score (47.90909090909091)
Win/lose count 27.5/2.0. Average score (46.041666666666664)
Win/lose count 64.0/5.0. Average score (47.03846153846154)
Win/lose count 32.5/1.0. Average score (45.92857142857143)
Win/lose count 33.5/0. Average score (45.1)
Win/lose count 47.5/1.0. Average score (45.1875)
Win/lose count 30.0/1.0. Average score (44.23529411764706)
Win/lose count 33.5/1.0. Average score (43.583333333333336)
Win/lose count 55.5/1.0. Average score (44.1578947368421)
Win/los

We get poorer results when we disable the exploration (43.2 vs 49.1). This can be explained because random exploration is helpful to get out of an infinite loop and help to explore more parts of the map. (and this is very important with a high temperature).

***

The algorithm tends to not explore the map which can be an issue. We propose two ideas in order to encourage exploration:
1. Incorporating a decreasing $\epsilon$-greedy exploration. You can use the method ```set_epsilon```
2. Append via the environment a new state that describes if a cell has been visited or not

***
__Question 10__ Design a new ```train_explore``` function and environment class ```EnvironmentExploring``` to tackle the issue of exploration.



In [37]:
def train_explore(agent, env, epoch, prefix=''):
    # Number of won games
    score = 0
    loss = 0
    
    current_epsilon = 0.2
    final_eps = 0.01
    
    incr = (current_epsilon - final_eps) / epoch

    for e in range(epoch):
        
        current_epsilon -= incr
        agent.set_epsilon(current_epsilon)
        
        # At each epoch, we restart to a fresh game and get the initial state
        state = env.reset()
        # This assumes that the games will terminate
        game_over = False

        win = 0
        lose = 0

        while not game_over:
            action = agent.act(state)
            prev_state = state
            state, reward, game_over = env.act(action, train=True)

            # Update the counters
            if reward > 0:
                win = win + reward
            if reward < 0:
                lose = lose -reward

            # Apply the reinforcement strategy
            loss = agent.reinforce(prev_state, state,  action, reward, game_over)

        # Save as a mp4
        if e % 10 == 0:
            env.draw(prefix+str(e))

        # Update stats
        score += win-lose

        print("Epoch {:03d}/{:03d} | Loss {:.4f} | Win/lose count {}/{} ({})"
              .format(e, epoch, loss, win, lose, win-lose))
        agent.save(name_weights=prefix+'model.h5',name_model=prefix+'model.json')
    
class EnvironmentExploring(object):
    def __init__(self, grid_size=10, max_time=500, temperature=0.1):
        grid_size = grid_size+4
        self.grid_size = grid_size
        self.max_time = max_time
        self.temperature = temperature

        #board on which one plays
        self.board = np.zeros((grid_size,grid_size))
        self.position = np.zeros((grid_size,grid_size))
        self.malus_position = np.zeros((grid_size,grid_size)).astype(int)

        # coordinate of the cat
        self.x = 0
        self.y = 1

        # self time
        self.t = 0

        self.scale=16

        self.to_draw = np.zeros((max_time+2, grid_size*self.scale, grid_size*self.scale, 3))


    def draw(self,e):
        skvideo.io.vwrite(str(e) + '.mp4', self.to_draw)

    def get_frame(self,t):
        b = np.zeros((self.grid_size,self.grid_size,3))+128
        b[self.board>0,0] = 256
        b[self.board < 0, 2] = 256
        b[self.x,self.y,:]=256
        b[-2:,:,:]=0
        b[:,-2:,:]=0
        b[:2,:,:]=0
        b[:,:2,:]=0
        
        b =  cv2.resize(b, None, fx=self.scale, fy=self.scale, interpolation=cv2.INTER_NEAREST)

        self.to_draw[t,:,:,:]=b


    def act(self, action, train=False):
        """This function returns the new state, reward and decides if the
        game ends."""

        self.get_frame(int(self.t))

        self.position = np.zeros((self.grid_size, self.grid_size))

        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1

        self.position[self.x, self.y] = 1
        if action == 0:
            if self.x == self.grid_size-3:
                self.x = self.x-1
            else:
                self.x = self.x + 1
        elif action == 1:
            if self.x == 2:
                self.x = self.x+1
            else:
                self.x = self.x-1
        elif action == 2:
            if self.y == self.grid_size - 3:
                self.y = self.y - 1
            else:
                self.y = self.y + 1
        elif action == 3:
            if self.y == 2:
                self.y = self.y + 1
            else:
                self.y = self.y - 1
        else:
            RuntimeError('Error: action not recognized')
            
        reward = 0
        if train:
            reward += -self.malus_position[self.x, self.y]
        self.malus_position[self.x, self.y] = 0.1

        self.t = self.t + 1
        reward += self.board[self.x, self.y]
        self.board[self.x, self.y] = 0
        game_over = self.t > self.max_time
        state = np.concatenate(
            (
                self.malus_position.reshape(self.grid_size, self.grid_size, 1),
                self.board.reshape(self.grid_size, self.grid_size, 1),
                self.position.reshape(self.grid_size, self.grid_size,1)
            )
            , axis=2
        )
        
        state = state[self.x-2:self.x+3,self.y-2:self.y+3,:]

        return state, reward, game_over

    def reset(self):
        """This function resets the game and returns the initial state"""

        self.x = np.random.randint(3, self.grid_size-3, size=1)[0]
        self.y = np.random.randint(3, self.grid_size-3, size=1)[0]
        
        self.malus_position = np.zeros((self.grid_size, self.grid_size))

        bonus = 0.5 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        bonus = bonus.reshape(self.grid_size, self.grid_size)

        malus = -1.0 * np.random.binomial(1, self.temperature, size=self.grid_size**2)
        malus = malus.reshape(self.grid_size, self.grid_size)

        self.to_draw = np.zeros((self.max_time+2, self.grid_size*self.scale, self.grid_size*self.scale, 3))


        malus[bonus>0]=0

        self.board = bonus + malus

        self.position = np.zeros((self.grid_size, self.grid_size))
        self.position[0:2,:]= -1
        self.position[:,0:2] = -1
        self.position[-2:, :] = -1
        self.position[-2:, :] = -1
        self.board[self.x,self.y] = 0
        self.t = 0

        state = np.concatenate(
            (
            self.malus_position.reshape(self.grid_size, self.grid_size, 1),
            self.board.reshape(self.grid_size, self.grid_size, 1),
            self.position.reshape(self.grid_size, self.grid_size,1)
            )
            , axis=2
        )

        state = state[self.x - 2:self.x + 3, self.y - 2:self.y + 3, :]
        return state

In [38]:
# Training
env = EnvironmentExploring(grid_size=size, max_time=T, temperature=0.3)
agent = DQN_CNN(size, lr=.1, epsilon = 0.2, memory_size=100000, batch_size = 32, n_state=3)
epochs_train = 50
train_explore(agent, env, epochs_train, prefix='cnn_train_explore')
HTML(display_videos('cnn_train_explore10.mp4'))

Epoch 000/050 | Loss 0.0049 | Win/lose count 4.0/21.700000000000045 (-17.700000000000045)
Epoch 001/050 | Loss 0.0093 | Win/lose count 2.5/22.900000000000066 (-20.400000000000066)
Epoch 002/050 | Loss 0.0093 | Win/lose count 6.5/22.200000000000056 (-15.700000000000056)
Epoch 003/050 | Loss 0.0079 | Win/lose count 3.5/23.90000000000009 (-20.40000000000009)
Epoch 004/050 | Loss 0.0078 | Win/lose count 4.0/20.80000000000003 (-16.80000000000003)
Epoch 005/050 | Loss 0.0087 | Win/lose count 8.5/19.700000000000014 (-11.200000000000014)
Epoch 006/050 | Loss 0.0053 | Win/lose count 5.0/21.800000000000054 (-16.800000000000054)
Epoch 007/050 | Loss 0.0059 | Win/lose count 1.0/21.600000000000044 (-20.600000000000044)
Epoch 008/050 | Loss 0.0061 | Win/lose count 5.0/25.60000000000008 (-20.60000000000008)
Epoch 009/050 | Loss 0.0100 | Win/lose count 5.0/20.60000000000003 (-15.60000000000003)
Epoch 010/050 | Loss 0.0050 | Win/lose count 5.5/26.300000000000082 (-20.800000000000082)
Epoch 011/050 | Lo

In [39]:
# Evaluation
print("evaluate CNN train_explore with epsilon=0.1")
agent.set_epsilon(0.1)
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

evaluate CNN train_explore with epsilon=0.1
Win/lose count 13.0/4.0. Average score (9.0)
Win/lose count 16.5/3.0. Average score (11.25)
Win/lose count 25.5/5.0. Average score (14.333333333333334)
Win/lose count 17.0/1.0. Average score (14.75)
Win/lose count 11.0/2.0. Average score (13.6)
Win/lose count 17.0/8.0. Average score (12.833333333333334)
Win/lose count 11.0/1.0. Average score (12.428571428571429)
Win/lose count 12.0/3.0. Average score (12.0)
Win/lose count 23.0/7.0. Average score (12.444444444444445)
Win/lose count 16.0/3.0. Average score (12.5)
Win/lose count 13.5/3.0. Average score (12.318181818181818)
Win/lose count 8.0/2.0. Average score (11.791666666666666)
Win/lose count 22.0/4.0. Average score (12.26923076923077)
Win/lose count 14.0/1.0. Average score (12.321428571428571)
Win/lose count 17.0/3.0. Average score (12.433333333333334)
Win/lose count 17.0/3.0. Average score (12.53125)
Win/lose count 13.5/3.0. Average score (12.411764705882353)
Win/lose count 17.5/1.0. Averag

We get excellent results by adding these two tricks (12.7 vs 9.2). We can see in the videos that the agent is exploring much more the map and therefore is able to get bigger rewards.

Below we try again to disable the exploration.

In [40]:
# Evaluation
print("evaluate CNN train_explore with epsilon=0.0")
agent.set_epsilon(0.)
test(agent, env, epochs_test, prefix='cnn_test_explore')
HTML(display_videos('cnn_test_explore10.mp4'))

evaluate CNN train_explore with epsilon=0.0
Win/lose count 27.5/1.0. Average score (26.5)
Win/lose count 19.0/0. Average score (22.75)
Win/lose count 8.0/0. Average score (17.833333333333332)
Win/lose count 15.5/2.0. Average score (16.75)
Win/lose count 8.5/0. Average score (15.1)
Win/lose count 13.5/0. Average score (14.833333333333334)
Win/lose count 19.0/0. Average score (15.428571428571429)
Win/lose count 13.0/0. Average score (15.125)
Win/lose count 4.0/0. Average score (13.88888888888889)
Win/lose count 10.0/0. Average score (13.5)
Win/lose count 13.5/1.0. Average score (13.409090909090908)
Win/lose count 10.0/0. Average score (13.125)
Win/lose count 15.0/1.0. Average score (13.192307692307692)
Win/lose count 18.5/0. Average score (13.571428571428571)
Win/lose count 6.0/0. Average score (13.066666666666666)
Win/lose count 14.5/2.0. Average score (13.03125)
Win/lose count 7.0/0. Average score (12.676470588235293)
Win/lose count 6.0/0. Average score (12.305555555555555)
Win/lose co

We get better results with `epsilon=0.` than `epsilon=0.1`. This is probably because the randomness brought by the non zero `epsilon` helps to get out of deadlock situations and therefore explore the entire map.

***
***
__BONUS question__ Use the expert DQN from the previous question to generate some winning games. Train a model that mimicks its behavior. Compare the performances.

***